<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_metric_kernel_final_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
quantum_metric_kernel_final.py

End-to-end 4-bit parity classification:

1) Analytic CNOT cascade (100% accuracy)
2) Variational QNN with 5-fold CV
3) Quantum metric learning with a trainable feature map:
   - Deeper ring-entangled ansatz (4 layers)
   - Centered kernel–target alignment objective
   - Adam optimizer (no QNG) for smooth gradients
   - 100 training steps, lr=0.5
   - Final SVM evaluation with 5-fold CV
   - Kernel heatmap visualization

Dependencies:
  pip install pennylane numpy scikit-learn matplotlib
"""

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# Hyperparameters
# ------------------------------------------------------------------------------
n_bits     = 4       # parity on 4‐bit strings
layers     = 4       # number of entangling layers in feature map
n_steps    = 100     # training steps for metric learning
lr_metric  = 0.5     # learning rate for Adam optimizer
svm_C      = 1.0     # SVM regularization parameter
qnn_epochs = 30      # epochs for variational QNN
qnn_lr     = 0.4     # learning rate for QNN

# ------------------------------------------------------------------------------
# 1) Generate parity dataset
# ------------------------------------------------------------------------------
X = pnp.array(list(itertools.product([0, 1], repeat=n_bits)))
y = pnp.array(np.sum(X, axis=1) % 2)  # labels in {0,1}

# ------------------------------------------------------------------------------
# 2) Analytic CNOT cascade
# ------------------------------------------------------------------------------
dev_exact = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_exact)
def analytic_circuit(x):
    # prepare input bits
    for i, bit in enumerate(x):
        if bit:
            qml.PauliX(wires=i)
    # cascade CNOT onto ancilla
    for i in range(n_bits):
        qml.CNOT(wires=[i, n_bits])
    # measure Z on ancilla: +1 ➞ even, -1 ➞ odd
    return qml.expval(qml.PauliZ(wires=n_bits))

preds_analytic = [0 if analytic_circuit(x) > 0 else 1 for x in X]
acc_analytic   = accuracy_score(y, preds_analytic)
print(f"Analytic CNOT Cascade Accuracy: {acc_analytic:.3f}")

# ------------------------------------------------------------------------------
# 3) Variational QNN with 5‐fold CV
# ------------------------------------------------------------------------------
dev_qnn = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_qnn, interface="autograd")
def qnn_circuit(params, x):
    # encode each bit with RY(pi) if bit==1
    for i, bit in enumerate(x):
        if bit:
            qml.RY(np.pi, wires=i)
    # trainable RY rotations
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    # ladder entanglement
    for i in range(n_bits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.expval(qml.PauliZ(wires=n_bits - 1))

# prepare signed labels for regression loss
y_signed = 1 - 2 * y  # {0,1}→{+1,-1}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
var_scores = []

for train_idx, test_idx in kf.split(X, y):
    # init parameters
    params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
    opt    = qml.GradientDescentOptimizer(stepsize=qnn_lr)
    # train
    for _ in range(qnn_epochs):
        def cost(p):
            preds = [qnn_circuit(p, X[i]) for i in train_idx]
            return pnp.mean((pnp.array(preds) - y_signed[train_idx]) ** 2)
        params = opt.step(cost, params)
    # evaluate
    preds_test = [qnn_circuit(params, X[i]) for i in test_idx]
    bits       = [0 if p > 0 else 1 for p in preds_test]
    var_scores.append(accuracy_score(y[test_idx], bits))

print(
    f"Variational QNN CV Accuracy: "
    f"{np.mean(var_scores):.3f} ± {np.std(var_scores):.3f}"
)

# ------------------------------------------------------------------------------
# 4) Quantum metric learning: trainable feature map + centered alignment
# ------------------------------------------------------------------------------

dev_feat = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_feat, interface="autograd")
def feature_map(params, x):
    # angle‐embedding of input bits
    qml.templates.AngleEmbedding(x, wires=range(n_bits))
    # rotate+ring-entangle layers
    for layer in range(layers):
        for wire in range(n_bits):
            phi, theta, omega = params[layer, wire]
            qml.Rot(phi, theta, omega, wires=wire)
        for i in range(n_bits):
            qml.CNOT(wires=[i, (i + 1) % n_bits])
    return qml.state()

def build_kernel_matrix(params):
    # stack statevectors for all inputs
    states = pnp.stack([feature_map(params, x) for x in X])
    re     = pnp.real(states)
    im     = pnp.imag(states)
    # compute overlaps via matrix mult
    real_ov = re @ re.T + im @ im.T
    imag_ov = re @ im.T - im @ re.T
    return real_ov**2 + imag_ov**2

def centered_alignment(K):
    N = K.shape[0]
    H = pnp.eye(N) - (1 / N) * pnp.ones((N, N))
    Kc = H @ K @ H
    y_s = 1 - 2 * y
    T   = pnp.outer(y_s, y_s)
    num = pnp.sum(Kc * T)
    den = pnp.linalg.norm(Kc) * pnp.linalg.norm(T)
    return num / den

# initialize parameters in [0, 2π]
params_k = pnp.array(
    np.random.rand(layers, n_bits, 3) * 2 * np.pi,
    requires_grad=True
)
opt_k = qml.AdamOptimizer(stepsize=lr_metric)

print("\nTraining feature‐map via centered alignment:")
for step in range(n_steps):
    def loss(p):
        K = build_kernel_matrix(p)
        return -centered_alignment(K)
    params_k, cost = opt_k.step_and_cost(loss, params_k)
    if step % 10 == 0:
        align = centered_alignment(build_kernel_matrix(params_k))
        print(f" Step {step:>3}: alignment = {align:.4f}")

# ------------------------------------------------------------------------------
# 5) Final SVM CV with learned kernel
# ------------------------------------------------------------------------------
K_final = np.array(build_kernel_matrix(params_k))
ker_scores = []

for tr, te in kf.split(K_final, y):
    svc = SVC(kernel="precomputed", C=svm_C)
    svc.fit(K_final[np.ix_(tr, tr)], y[tr])
    preds = svc.predict(K_final[np.ix_(te, tr)])
    ker_scores.append(accuracy_score(y[te], preds))

print(
    f"\nQuantum‐Metric Kernel CV Accuracy: "
    f"{np.mean(ker_scores):.3f} ± {np.std(ker_scores):.3f}"
)

# ------------------------------------------------------------------------------
# 6) Plot the trained kernel matrix
# ------------------------------------------------------------------------------
plt.figure(figsize=(4, 4))
plt.title("Learned Kernel Matrix (|⟨ψ_i|ψ_j⟩|²)")
plt.imshow(K_final, cmap="viridis", origin="lower")
plt.colorbar(label="Kernel value")
plt.xlabel("Index j")
plt.ylabel("Index i")
plt.tight_layout()
plt.show()